In [2]:
import pandas as pd 
import ast
import json
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
from sklearn.neighbors import NearestNeighbors

In [3]:
data = pd.read_csv('c:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/movie_tmdb.csv' , sep=',')
movie = pd.read_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/movie.csv', sep=',')

""" Changement des formats facilitant les traitements """

# Convertir les chaînes en listes de dictionnaires
data['spoken_languages'] = data['spoken_languages'].apply(ast.literal_eval)
# Convertir la chaîne en liste python avec ast.literal_eval (plus sûr que json.loads si la chaîne n'est pas en format JSON strict)
data['production_countries'] = data['production_countries'].apply(ast.literal_eval)

# Trier les genres qu'il nous faut
# Trier le spoken_language en fr ou French ou Fr
# Changer le format de la realese_date en date
# Changer le nom de imdb_id en tconst
# Rajouter une colonne décennie
# Rajouter les informations sur les acteurs, actrices et directeurs avec l'api
# Trier sur les meilleurs films top 5 %  des films par décennie
# Voir lequel des vite_average et vote_count gardé
# Merger avec movie_csv
# Détermination des types de colonnes 
# data_fr['spoken_languages'].apply(type).value_counts()
# Détermination des types de colonnes 
# movie_tmdb['production_countries'].apply(type).value_counts()

In [4]:
#convertir les colonnes (genres, actors, actresses, directors) en liste
movie['genres'] = movie['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actors'] = movie['actors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actresses'] = movie['actresses'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['directors'] = movie['directors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

In [5]:
data_fr = data.copy()

# Filtrer pour ne garder que les films exclusivement en français
data_fr = data_fr[data_fr['spoken_languages'].apply(lambda langs: any(lang.get('iso_639_1') == 'fr' for lang in langs))]

# Supprimer les colonnes inutiles
data_fr = data_fr.drop(columns=['adult','backdrop_path','belongs_to_collection','budget', 'homepage','production_companies',
                                'revenue','origin_country','tagline','video','title'])

""" Realease date formatage et création de 2 colonnes"""
#Changement du format de la released_date 
data_fr['release_date'] = pd.to_datetime(data_fr['release_date'])

# Rajout de la colonne decennie et start year
data_fr['startYear'] = data_fr['release_date'].dt.year

# Créer la colonne décennie
data_fr['decennie'] = (data_fr['startYear'] // 10) * 10

""" Genre Formatage """
# Convertir les chaînes en listes de dictionnaires
data_fr['genres'] = data_fr['genres'].apply(ast.literal_eval)

# Liste des genres à exclure
genres_exclus = {'Documentaire', 'Musique', 'Téléfilm'}

movie_tmdb = data_fr.copy() 

# Filtrer pour supprimer les films contenant ces genres
movie_tmdb = movie_tmdb[~movie_tmdb['genres'].apply(lambda genre_list: any(g['name'] in genres_exclus for g in genre_list))]

# Changement du nom imdb_id en tconst
movie_tmdb.rename(columns={'imdb_id': 'tconst'}, inplace=True)

""" Réinitialisation de l'index """
# Réinitialiser les index
movie_tmdb.reset_index(drop=True, inplace=True)

# Affichage
movie_tmdb

,genres,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,actors,actresses,directors,startYear,decennie
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",575265,tt9603208,en,Mission: Impossible - The Final Reckoning,Ethan Hunt se rend à Londres avec son équipe d...,267.7424,/sOvTMbTHmp13LWYvgMXDvbY1auX.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2025-05-15,163,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7.100,179,"['Tom Cruise', 'Ving Rhames', 'Simon Pegg', 'E...","['Hayley Atwell', 'Pom Klementieff', 'Janet Mc...",['Christopher McQuarrie'],2025,2020
1,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1144430,tt29768333,fr,Balle perdue 3,"Dans un dernier opus chargé en adrénaline, le ...",145.5038,/xccJs9vGCEvch1H9SIMSjNqrLKU.jpg,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",2025-05-06,111,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,6.799,127,"['Alban Lenoir', 'Nicolas Duvauchelle', 'Gérar...","['Stéfi Celma', 'Pascale Arbillot', 'Anne Serra']",['Guillaume Pierret'],2025,2020
2,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",939243,tt18259086,en,Sonic the Hedgehog 3,"Sonic, Knuckles et Tails sont à nouveau réunis...",73.6897,/gERwLGTa6JGN4qXjkip13eDaxy1.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2024-12-19,100,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7.715,2587,"['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","[""Colleen O'Shaughnessey"", 'Tika Sumpter', 'Kr...",['Jeff Fowler'],2024,2020
3,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",1064486,tt23770030,en,Memoir of a Snail,"La vie de Grace Pudel, petite fille solitaire,...",34.0607,/oYpq6XFoubsezCt1o28TwnraD0a.jpg,"[{'iso_3166_1': 'AU', 'name': 'Australia'}]",2024-10-17,94,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,8.068,505,"['Kodi Smit-McPhee', 'Dominique Pinon', 'Tony ...","['Sarah Snook', 'Jacki Weaver', 'Magda Szubans...",['Adam Elliot'],2024,2020
4,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",1087891,tt0899043,en,The Amateur,"Un cryptographe de la CIA qui, après la mort t...",27.0158,/l2EmTzKN7qZvAW6p7JfH6v09tAO.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2025-04-09,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,6.651,259,"['Rami Malek', 'Holt McCallany', 'Danny Sapani...","['Rachel Brosnahan', 'Julianne Nicholson', 'Ca...",['James Hawes'],2025,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,"[{'id': 18, 'name': 'Drame'}, {'id': 10749, 'n...",1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,"[{'iso_3166_1': 'FR', 'name': 'France'}]",2025-03-05,101,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,6.600,5,"['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous'],2025,2020
432,"[{'id': 35, 'name': 'Comédie'}, {'id': 18, 'na...",892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,"[{'iso_3166_1': 'FR', 'name': 'France'}]",2024-08-11,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,5.500,4,"['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco'],2024,2020
433,"[{'id': 18, 'name': 'Drame'}, {'id': 35, 'name...",1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",2025-11-12,108,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,0.000,0,"['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine'],2025,2020
434,"[{'id': 18, 'name': 'Drame'}]",1262533,tt16893002,fr,Le silence d

In [6]:
movie_tmdb['genres'] = movie_tmdb['genres'].apply(
    lambda genre_list: [g.get('name') for g in genre_list if isinstance(g, dict) and g.get('name')]
)

movie_tmdb['spoken_languages'] = movie_tmdb['spoken_languages'].apply(
    lambda langs: [l.get('iso_639_1') for l in langs if isinstance(l, dict) and l.get('iso_639_1')]
)

movie_tmdb['production_countries'] = movie_tmdb['production_countries'].apply(lambda countries: [c['name'] for c in countries])
movie_tmdb

,genres,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,actors,actresses,directors,startYear,decennie
0,"[Action, Aventure, Thriller]",575265,tt9603208,en,Mission: Impossible - The Final Reckoning,Ethan Hunt se rend à Londres avec son équipe d...,267.7424,/sOvTMbTHmp13LWYvgMXDvbY1auX.jpg,[United States of America],2025-05-15,163,"[en, fr, iu]",Released,7.100,179,"['Tom Cruise', 'Ving Rhames', 'Simon Pegg', 'E...","['Hayley Atwell', 'Pom Klementieff', 'Janet Mc...",['Christopher McQuarrie'],2025,2020
1,"[Action, Crime, Thriller, Drame]",1144430,tt29768333,fr,Balle perdue 3,"Dans un dernier opus chargé en adrénaline, le ...",145.5038,/xccJs9vGCEvch1H9SIMSjNqrLKU.jpg,"[Belgium, France]",2025-05-06,111,[fr],Released,6.799,127,"['Alban Lenoir', 'Nicolas Duvauchelle', 'Gérar...","['Stéfi Celma', 'Pascale Arbillot', 'Anne Serra']",['Guillaume Pierret'],2025,2020
2,"[Action, Science-Fiction, Comédie, Familial]",939243,tt18259086,en,Sonic the Hedgehog 3,"Sonic, Knuckles et Tails sont à nouveau réunis...",73.6897,/gERwLGTa6JGN4qXjkip13eDaxy1.jpg,"[United States of America, Japan]",2024-12-19,100,"[en, fr, ja, es]",Released,7.715,2587,"['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","[""Colleen O'Shaughnessey"", 'Tika Sumpter', 'Kr...",['Jeff Fowler'],2024,2020
3,"[Animation, Drame, Comédie]",1064486,tt23770030,en,Memoir of a Snail,"La vie de Grace Pudel, petite fille solitaire,...",34.0607,/oYpq6XFoubsezCt1o28TwnraD0a.jpg,[Australia],2024-10-17,94,"[fr, en]",Released,8.068,505,"['Kodi Smit-McPhee', 'Dominique Pinon', 'Tony ...","['Sarah Snook', 'Jacki Weaver', 'Magda Szubans...",['Adam Elliot'],2024,2020
4,"[Thriller, Action]",1087891,tt0899043,en,The Amateur,"Un cryptographe de la CIA qui, après la mort t...",27.0158,/l2EmTzKN7qZvAW6p7JfH6v09tAO.jpg,[United States of America],2025-04-09,123,"[en, fr, ru]",Released,6.651,259,"['Rami Malek', 'Holt McCallany', 'Danny Sapani...","['Rachel Brosnahan', 'Julianne Nicholson', 'Ca...",['James Hawes'],2025,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,"[Drame, Romance, Comédie]",1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05,101,[fr],Released,6.600,5,"['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous'],2025,2020
432,"[Comédie, Drame]",892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11,83,"[en, fr, it]",Released,5.500,4,"['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco'],2024,2020
433,"[Drame, Comédie]",1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12,108,[fr],Released,0.000,0,"['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine'],2025,2020
434,[Drame],1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01,98,[fr],Released,5.000,1,[],"['Laëtitia EÏDO', 'Marie-Bénédicte Cazeneuve']",['Aly Yeganeh'],2024,2020


In [7]:
# Changement du nom pour faciliter la concaténation
movie.rename(columns={'decennie_x': 'decennie'}, inplace=True)
movie_tmdb.rename(columns={'runtime': 'runtimeMinutes'}, inplace=True)

# Concaténation et non pas une merge car on rajoute des lignes en plus afin d'enrichir notre dataframe
films =  pd.concat([movie, movie_tmdb], ignore_index=True)

# Suppression des doublons
films.drop_duplicates(subset='id', inplace=True)

# changer le Format de released_date en date time
films['release_date'] = pd.to_datetime(films['release_date'], errors='coerce')

# Start year à partir de 1960
films = films[films['startYear'] >= 1960]

# Filtrer la note à partir de 5.8
#films = films[films['vote_average'] >= 5.8]

# Reset de l'index 
films.reset_index(drop=True, inplace=True)

films

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,status,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,actors,actresses,directors
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.7700,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",Released,6.700,41,1960.0,1962.0,110.0,"[Drama, Romance]","[Jean-Pierre Léaud, Patrick Auffay, François...","[Marie-France Pisier, Rosy Varte, Cristina G...","[Shintarô Ishihara, Marcel Ophüls, Renzo Ros..."
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.5560,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",Released,7.970,15748,2000.0,2003.0,111.0,"[Action, Crime, Thriller]","[David Carradine, Michael Madsen, Shin'ichi ...","[Uma Thurman, Daryl Hannah, Lucy Liu, Vivic...",[Quentin Tarantino]
2,75,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.0510,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",Released,6.394,4864,1990.0,1996.0,106.0,"[Comedy, Sci-Fi]","[Jack Nicholson, Jack Nicholson, Pierce Bros...","[Sarah Jessica Parker, Annette Bening, Glenn...",NaN
3,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.3250,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",Released,7.968,3547,1990.0,1995.0,101.0,"[Comedy, Drama, Romance]","[Ethan Hawke, Hanno Pöschl, Karl Bruckschwai...","[Julie Delpy, Andrea Eckert, Erni Mangold]",[Richard Linklater]
4,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.1840,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",Released,7.818,2943,2000.0,2004.0,80.0,"[Drama, Romance]","[Ethan Hawke, Vernon Dobtcheff, Rodolphe Pau...","[Julie Delpy, Louise Lemoine Torrès, Mariane...",[Richard Linklater]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9708,1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05,[fr],Released,6.600,5,2020.0,2025.0,101.0,"[Drame, Romance, Comédie]","['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous']
9709,892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11,"[en, fr, it]",Released,5.500,4,2020.0,2024.0,83.0,"[Comédie, Drame]","['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco']
9710,1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12,[fr],Released,0.000,0,2020.0,2025.0,108.0,"[Drame, Comédie]","['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine']
9711,1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01,[fr],Released,5.000,1,2020.0,2024.0,98.0,[Drame],[],"['Laëtitia EÏDO', 'Marie-Bénédicte Cazeneuve']",['Aly Yeganeh']


                                                    Machine Learning

In [8]:
# Nettoyage des lignes manquantes
films = films.dropna()
films = films.copy()
films.reset_index(drop=True, inplace=True)
# nouveau dataframe avec uniquement les deux premiers acteurs, actrices et un réalisateur 
films[['acteurs_1','acteurs_2']] = films['actors'].apply(lambda x: pd.Series(x[:2]))
films['actrices'] = films['actresses'].apply(lambda x: x[0] if len(x) > 0 else None) 
films['realisateurs'] = films['directors'].apply(lambda x: x[0] if len(x) > 0 else None) 

# Suppression des colonnes actors, acrtress et directors
films = films.drop(columns=['actors', 'actresses', 'directors'])    

films


,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.7700,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",...,6.700,41,1960.0,1962.0,110.0,"[Drama, Romance]",Jean-Pierre Léaud,Patrick Auffay,Marie-France Pisier,Shintarô Ishihara
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.5560,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",...,7.970,15748,2000.0,2003.0,111.0,"[Action, Crime, Thriller]",David Carradine,Michael Madsen,Uma Thurman,Quentin Tarantino
2,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.3250,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",...,7.968,3547,1990.0,1995.0,101.0,"[Comedy, Drama, Romance]",Ethan Hawke,Hanno Pöschl,Julie Delpy,Richard Linklater
3,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.1840,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",...,7.818,2943,2000.0,2004.0,80.0,"[Drama, Romance]",Ethan Hawke,Vernon Dobtcheff,Julie Delpy,Richard Linklater
4,101,tt0110413,en,Léon: The Professional,"Léon, the top hit man in New York, has earned ...",44.7010,/yI6X2cCM5YPJtxMhUd3dPGqDAhw.jpg,"['US', 'FR']",1994-09-14,"['en', 'fr', 'it']",...,8.300,13202,1990.0,1994.0,110.0,"[Action, Crime, Drama]",Jean Reno,Gary Oldman,Natalie Portman,Luc Besson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8566,1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05,[fr],...,6.600,5,2020.0,2025.0,101.0,"[Drame, Romance, Comédie]",[',NaN,[,[
8567,892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11,"[en, fr, it]",...,5.500,4,2020.0,2024.0,83.0,"[Comédie, Drame]",[',NaN,[,[
8568,1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12,[fr],...,0.000,0,2020.0,2025.0,108.0,"[Drame, Comédie]",[',NaN,[,[
8569,1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01,[fr],...,5.000,1,2020.0,2024.0,98.0,[Drame],[],NaN,[,[


In [9]:
# Exportation du dataframe en csv
films.to_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/films.csv', index=False)
films

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.7700,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",...,6.700,41,1960.0,1962.0,110.0,"[Drama, Romance]",Jean-Pierre Léaud,Patrick Auffay,Marie-France Pisier,Shintarô Ishihara
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.5560,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",...,7.970,15748,2000.0,2003.0,111.0,"[Action, Crime, Thriller]",David Carradine,Michael Madsen,Uma Thurman,Quentin Tarantino
2,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.3250,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",...,7.968,3547,1990.0,1995.0,101.0,"[Comedy, Drama, Romance]",Ethan Hawke,Hanno Pöschl,Julie Delpy,Richard Linklater
3,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.1840,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",...,7.818,2943,2000.0,2004.0,80.0,"[Drama, Romance]",Ethan Hawke,Vernon Dobtcheff,Julie Delpy,Richard Linklater
4,101,tt0110413,en,Léon: The Professional,"Léon, the top hit man in New York, has earned ...",44.7010,/yI6X2cCM5YPJtxMhUd3dPGqDAhw.jpg,"['US', 'FR']",1994-09-14,"['en', 'fr', 'it']",...,8.300,13202,1990.0,1994.0,110.0,"[Action, Crime, Drama]",Jean Reno,Gary Oldman,Natalie Portman,Luc Besson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8566,1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05,[fr],...,6.600,5,2020.0,2025.0,101.0,"[Drame, Romance, Comédie]",[',NaN,[,[
8567,892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11,"[en, fr, it]",...,5.500,4,2020.0,2024.0,83.0,"[Comédie, Drame]",[',NaN,[,[
8568,1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12,[fr],...,0.000,0,2020.0,2025.0,108.0,"[Drame, Comédie]",[',NaN,[,[
8569,1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01,[fr],...,5.000,1,2020.0,2024.0,98.0,[Drame],[],NaN,[,[


In [10]:
# Custom transformer for MultiLabelBinarizer
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_
    
# Define the ColumnTransformer and the pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('acteurs', OneHotEncoder(), ['acteurs_1', 'acteurs_2','actrices']),
        ('réalisateur', OneHotEncoder(), ['realisateurs']),
        ('status', OneHotEncoder (), ['status']),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genres'),
        ('annee', OrdinalEncoder(), ['decennie']),
        ('note', 'passthrough', ['vote_average'])
    ]
)

# Create the pipeline with preprocessing
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

processed_data = pipeline.fit_transform(films)

from scipy.sparse import issparse

if issparse(processed_data):
    processed_data = processed_data.toarray()

df_processed = pd.DataFrame(
    processed_data.toarray() if issparse(processed_data) else processed_data,
    columns=pipeline.named_steps['preprocessor'].get_feature_names_out()
)
#df_processed = pd.concat([films["id"], df_processed], axis=1)
#df_processed

k= 5
nn_model = NearestNeighbors(n_neighbors=k+1, algorithm='auto', metric='euclidean')
nn_model.fit(df_processed)


# Exemple de recommandation pour un item
def recommend_similar_items(index: int, data: pd.DataFrame=df_processed, model: NearestNeighbors=nn_model, original_data: pd.DataFrame=films, n_neighbors: int=3) -> tuple[list[float], list[float]]:
    """
    Trouve les n éléments les plus proches pour un élément donné.

    Args:
        index: Index de l'élément de référence.
        data: Données transformées utilisées pour KNN.
        model: Modèle KNN pré-entraîné.
        original_data: Données originales (pour affichage).
        n_neighbors: Nombre de voisins à recommander.

    Returns:
        voisins: Indices et distances des voisins.
    """
    

    _, indices = model.kneighbors(data, n_neighbors=n_neighbors) # _ nous permet d'ignorer la variable distance
    print(f"Recommandations pour l'élément {index}:\n")
    display(original_data.iloc[indices[index][0:]]) # On veut afficher uniquement les voisins et pas le point de référence avec cad
                              #(si je fais un affichage des voisins de l index 0 je veux pas le point 0 qui s affiche)

recommend_similar_items(index=44, n_neighbors=11)


Recommandations pour l'élément 44:



,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
44,597,tt0120338,en,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,108.782,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,['US'],1997-11-18,"['en', 'fr', 'de', 'sv', 'it', 'ru']",...,7.892,22840,1990.0,1997.0,194.0,"[Drama, Romance]",Leonardo DiCaprio,Billy Zane,Kate Winslet,James Cameron
188,2767,tt0101318,fr,Les Amants du Pont-Neuf,"Set against Paris' oldest bridge, the Pont Neu...",11.182,/un81oln6mAv66sqnkgPSpWONUqP.jpg,['FR'],1991-10-16,['fr'],...,7.422,258,1990.0,1991.0,125.0,"[Drama, Romance]",Denis Lavant,Klaus-Michael Grüber,Juliette Binoche,Leos Carax
1603,739936,tt0304040,fr,French Lolita,"Paris, the city of love. Not for our young her...",6.135,/ydnX6nKNiJ7ESGi4nDT2YR4sTCK.jpg,['FR'],1998-02-24,['fr'],...,7.400,4,1990.0,1998.0,82.0,"[Drama, Romance]",Richard Sun,NaN,Cécile Fleury,Pierre B. Reinhard
3010,415823,tt0234614,fr,Romamor,A French director in Berlin creates a film let...,0.710,/g4WOnW4N3mqV7IZvWJWlhIwXYHr.jpg,['FR'],1992-01-01,['fr'],...,7.200,4,1990.0,1992.0,90.0,"[Drama, Romance]",Joseph Morder,NaN,Françoise Michaud,Joseph Morder
499,10241,tt0104008,fr,Conte d'hiver,Felicie and Charles have a whirlwind holiday r...,5.983,/uHmgUZXl7qkgu97E5ruDetLyDPt.jpg,['FR'],1992-01-29,['fr'],...,7.127,118,1990.0,1992.0,114.0,"[Drama, Romance]",Frédéric van den Driessche,Michel Voletti,Charlotte Véry,Éric Rohmer
6155,121895,tt0099341,fr,Daddy Nostalgie,"A half English, half French screenwriter visit...",1.730,/wp6wEQni9T2vUZETxBLCU0DQVRy.jpg,['FR'],1990-09-05,"['fr', 'en']",...,7.100,16,1990.0,1990.0,105.0,"[Drama, Romance]",Dirk Bogarde,NaN,Jane Birkin,Bertrand Tavernier
7925,65927,tt0105132,fr,La petite amie d'Antonio,It's a simple love story. Antonio likes Claudi...,0.870,/dLOO8uYWwa8OGyn8LZWB4RKq2sr.jpg,['FR'],1992-10-28,['fr'],...,7.000,1,1990.0,1992.0,105.0,"[Drama, Romance]",Sergi López,Guy-Pierre Mineur,Hélène Foubert,Manuel Poirier
5154,70270,tt0119113,fr,La Femme défendue,This movie is foremost distinguished by the us...,3.023,/lllbjY8mv3z9SqaJXIYAwycqhbs.jpg,['FR'],1997-09-24,['fr'],...,7.000,21,1990.0,1997.0,100.0,"[Drama, Romance]",Philippe Harel,Julien Niedergang,Isabelle Carré,Philippe Harel
4190,47333,tt0116684,en,Jane Eyre,Jane Eyre is an orphan cast out as a young gir...,10.168,/1NOWgJALHZWod4yv2SEaOxSp6w4.jpg,"['FR', 'IT', 'GB', 'US']",1996-02-09,"['fr', 'en']",...,6.927,193,1990.0,1996.0,112.0,"[Drama, Romance]",William Hurt,Nic Knight,Charlotte Gainsbourg,Franco Zeffirelli
3,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.184,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",...,7.818,2943,2000.0,2004.0,80.0,"[Drama, Romance]",Ethan Hawke,Vernon Dobtcheff,Julie Delpy,Richard Linklater


# Même modèle mais avec des paramètres différents 

In [11]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import issparse

# ---------------------- Custom Transformer ----------------------
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return [f"genres_{cls}" for cls in self.mlb.classes_]

# ---------------------- Prétraitement Optimisé ----------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('acteurs', OneHotEncoder(), ['acteurs_1', 'acteurs_2', 'actrices']),
        ('realisateur', OneHotEncoder(), ['realisateurs']),
        ('status', OneHotEncoder(), ['status']),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genres'),
        ('annee', OrdinalEncoder(), ['decennie']),
        ('note', 'passthrough', ['vote_average']),
        ('popularite', StandardScaler(), ['popularity']),
        ('duree', StandardScaler(), ['runtimeMinutes'])
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# ---------------------- Chargement des données ----------------------
films = pd.read_csv("C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/films.csv")  # Chargez vos données ici

# Vérifiez s'il existe déjà un fichier traité
try:
    df_processed = joblib.load('processed_films.pkl')
    nn_model = joblib.load('nn_model.pkl')
    distances_all, indices_all = joblib.load('nn_distances.pkl')
except FileNotFoundError:
    processed_data = pipeline.fit_transform(films)
    if issparse(processed_data):
        processed_data = processed_data.toarray()
    columns = pipeline.named_steps['preprocessor'].get_feature_names_out()
    df_processed = pd.DataFrame(processed_data, columns=columns)

    # Sauvegarde
    joblib.dump(df_processed, 'processed_films.pkl')

    nn_model = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
    nn_model.fit(df_processed)
    joblib.dump(nn_model, 'nn_model.pkl')

    distances_all, indices_all = nn_model.kneighbors(df_processed)
    joblib.dump((distances_all, indices_all), 'nn_distances.pkl')

# ---------------------- Fonction de recommandation ----------------------
def recommend_similar_items(index: int,
                             data: pd.DataFrame = df_processed,
                             model: NearestNeighbors = nn_model,
                             original_data: pd.DataFrame = films,
                             distances=distances_all,
                             indices=indices_all,
                             n_neighbors: int = 5,
                             sort_by: str = None,
                             min_popularity: float = None,
                             runtime_range: tuple = None) -> pd.DataFrame:

    neighbor_distances = distances[index][0:n_neighbors+1]
    neighbors_idx = indices[index][0:n_neighbors+1]

    recommendations = original_data.iloc[neighbors_idx].copy()
    recommendations['distance'] = neighbor_distances

    if min_popularity is not None:
        recommendations = recommendations[recommendations['popularity'] >= min_popularity]

    if runtime_range is not None:
        min_runtime, max_runtime = runtime_range
        recommendations = recommendations[
            recommendations['runtimeMinutes'].between(min_runtime, max_runtime)
        ]

    if sort_by and sort_by in recommendations.columns:
        recommendations = recommendations.sort_values(by=sort_by, ascending=False)

    print(f"\nRecommandations pour le film : {original_data.iloc[index]['original_title']}\n")
    return recommendations

# ---------------------- Exemple d'utilisation ----------------------
display(recommend_similar_items(
    index= 44,
    n_neighbors=10
))

#print(recommandations[['original_title', 'popularity', 'runtimeMinutes', 'distance']])


Recommandations pour le film : Titanic



,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs,distance
44,597,tt0120338,en,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,108.782,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,['US'],1997-11-18,"['en', 'fr', 'de', 'sv', 'it', 'ru']",...,22840,1990.0,1997.0,194.0,"['Drama', 'Romance']",Leonardo DiCaprio,Billy Zane,Kate Winslet,James Cameron,0.000000
36,497,tt0120689,en,The Green Mile,A supernatural tale set on death row in a Sout...,67.506,/o0lO84GI7qrG6XFvtsPOSV7CTNa.jpg,['US'],1999-12-10,"['fr', 'en']",...,15365,1990.0,1999.0,189.0,"['Crime', 'Drama', 'Fantasy']",Tom Hanks,Michael Clarke Duncan,Bonnie Hunt,Frank Darabont,4.645317
53,676,tt0213149,en,Pearl Harbor,The lifelong friendship between Rafe McCawley ...,42.180,/y8A0Cvp8WQmZ3bjbnsL53lY0dsC.jpg,['US'],2001-05-21,"['fr', 'en', 'ja']",...,5749,2000.0,2001.0,183.0,"['Action', 'Drama', 'Romance']",Ben Affleck,Josh Hartnett,Kate Beckinsale,Michael Bay,4.903365
54,680,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",59.938,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,['US'],1994-09-10,"['en', 'es', 'fr']",...,25111,1990.0,1994.0,154.0,"['Crime', 'Drama']",John Travolta,Samuel L. Jackson,Uma Thurman,Quentin Tarantino,5.090408
1407,842675,tt13539646,zh,流浪地球2,Humans built huge engines on the surface of th...,78.483,/pR858ihc6Ls9xohpdRJVjV787ml.jpg,['CN'],2023-01-22,"['zh', 'ja', 'fr', 'ru', 'en']",...,172,2020.0,2023.0,173.0,"['Action', 'Adventure', 'Drama']",Jing Wu,Yi Sha,Yanmanzi Zhu,Frant Gwo,5.281872
52,674,tt0330373,en,Harry Potter and the Goblet of Fire,When Harry Potter's name emerges from the Gobl...,119.827,/fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg,"['GB', 'US']",2005-11-16,"['en', 'fr']",...,18752,2000.0,2005.0,157.0,"['Adventure', 'Family', 'Fantasy']",Daniel Radcliffe,Rupert Grint,Emma Watson,Mike Newell,5.316319
840,16869,tt0361748,en,Inglourious Basterds,"In Nazi-occupied France during World War II, a...",49.227,/7sfbEnaARXDDhKm0CZ7D7uc2sbo.jpg,"['FR', 'DE', 'US']",2009-08-19,"['de', 'en', 'fr', 'it']",...,20148,2000.0,2009.0,153.0,"['Adventure', 'Drama', 'War']",Brad Pitt,Eli Roth,Diane Kruger,Quentin Tarantino,5.722951
2508,497828,tt7322224,en,Triangle of Sadness,A celebrity model couple are invited on a luxu...,85.194,/k9eLozCgCed5FGTSdHu0bBElAV8.jpg,"['DK', 'FR', 'DE', 'GR', 'MX', 'SE', 'TR', 'GB...",2022-09-18,"['el', 'de', 'sv', 'ak', 'en', 'tl', 'fr']",...,1256,2020.0,2022.0,147.0,"['Comedy', 'Drama']",Thobias Thorwid,Harris Dickinson,Charlbi Dean,Ruben Östlund,5.727215
456,9453,tt0080491,en,Caligola,"The perversion behind imperial Rome, the epic ...",64.895,/4N4l873TVreuaR4FVpvaL2gfQ0N.jpg,"['IT', 'US']",1979-08-14,['fr'],...,660,1970.0,1979.0,156.0,"['Drama', 'History']",Malcolm McDowell,Peter O'Toole,Helen Mirren,Tinto Brass,5.874478
6463,152584,tt2278871,fr,La Vie d'Adèle - Chapitres 1 et 2,"Adèle's life is changed when she meets Emma, a...",15.022,/kgUk1wti2cvrptIgUz0VTAtSF6w.jpg,"['BE', 'FR', 'ES']",2013-10-09,"['fr', 'en']",...,4142,2010.0,2013.0,180.0,"['Drama', 'Romance']",Salim Kechiouche,Aurélien Recoing,Léa Seydoux,Abdellatif Kechiche,5.878772


# Modèle prédisant un ou plus de films ayant au moins un des acteurs/actrices

In [12]:
def recommend_by_actors(index: int,
                        original_data: pd.DataFrame = films,
                        min_popularity: float = None,
                        decade: str = None,
                        sort_by: str = 'popularity',
                        top_n: int = 10) -> pd.DataFrame:
    def normalize_name(name):
        return name.strip().lower().replace("é", "e").replace("-", " ")

    # Nettoyage
    for col in ['acteurs_1', 'acteurs_2', 'actrices']:
        original_data[col] = original_data[col].fillna('').astype(str).str.strip().str.title()

    film_ref = original_data.iloc[index]
    acteurs_ref = set([
        film_ref['acteurs_1'],
        film_ref['acteurs_2'],
        film_ref['actrices']
    ])

    def has_common_actor(row):
        acteurs_row = set([
            row['acteurs_1'],
            row['acteurs_2'],
            row['actrices']
        ])
        return not acteurs_ref.isdisjoint(acteurs_row)

    filtered = original_data[original_data.index != index]
    filtered = filtered[filtered.apply(has_common_actor, axis=1)]

    if min_popularity is not None:
        filtered = filtered[filtered['popularity'] >= min_popularity]

    if decade is not None:
        filtered = filtered[filtered['decennie'] == decade]

    if sort_by and sort_by in filtered.columns:
        filtered = filtered.sort_values(by=sort_by, ascending=False)

    print(f"\nFilms avec {', '.join(acteurs_ref)} comme acteur(s)/actrice(s):\n")
    print(f"\nRecommandations avec au moins un acteur commun avec « {film_ref['original_title']} » :\n")
    return filtered.head(top_n)

display(recommend_by_actors(
    index= 44,
    min_popularity= None,
    top_n=10
))


Films avec Billy Zane, Kate Winslet, Leonardo Dicaprio comme acteur(s)/actrice(s):


Recommandations avec au moins un acteur commun avec « Titanic » :



,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
7447,281957,tt1663202,en,The Revenant,"In the 1820s, a frontiersman, Hugh Glass, sets...",34.120,/ji3ecJphATlVgWNY0B0RVXZizdf.jpg,"['CA', 'HK', 'TW', 'US']",2015-12-25,"['en', 'fr']",...,7.522,16748,2010.0,2015.0,156.0,"['Adventure', 'Drama', 'Western']",Leonardo Dicaprio,Tom Hardy,Melaw Nakehk'O,Alejandro G. Iñárritu
95,1581,tt0457939,en,The Holiday,"Two women, one from the United States and one ...",18.600,/6fbqG49Q7IWBWdyJ7asNTcNbnG6.jpg,['US'],2006-12-08,"['en', 'fr']",...,7.077,4452,2000.0,2006.0,136.0,"['Comedy', 'Romance']",Jude Law,Jack Black,Kate Winslet,Nancy Meyers
126,1907,tt0163978,en,The Beach,Twenty-something Richard travels to Thailand a...,17.314,/r6Sr0uSiVbQOvqoebZoy5rzlPvJ.jpg,"['GB', 'US']",2000-02-03,"['en', 'fr', 'hr', 'sv', 'th']",...,6.461,3917,2000.0,2000.0,119.0,"['Adventure', 'Drama', 'Romance']",Leonardo Dicaprio,Daniel York Loh,Tilda Swinton,Danny Boyle
494,10209,tt0136244,en,Hideous Kinky,"In 1972, disenchanted about the dreary convent...",8.107,/c1uprmmhS6LFt5GDcUuugcTBqDw.jpg,"['FR', 'GB']",1998-11-17,"['ar', 'en', 'fr']",...,6.188,77,1990.0,1998.0,98.0,"['Adventure', 'Drama', 'Romance']",Saïd Taghmaoui,Pierre Clémenti,Kate Winslet,Gillies MacKinnon


In [13]:
def normalize_name(name):
    return name.strip().lower().replace("é", "e").replace("è", "e").replace("-", " ").replace("_"," ").replace("ê"," ").replace("ô","o")

def get_films_by_actor(actor_name: str, original_data: pd.DataFrame = films) -> pd.DataFrame:
    # Normaliser les colonnes d'acteurs
    for col in ['acteurs_1', 'acteurs_2', 'actrices']:
        original_data[col] = original_data[col].fillna('').astype(str).apply(normalize_name)

    actor_name_normalized = normalize_name(actor_name)

    filtered = original_data[
        (original_data['acteurs_1'] == actor_name_normalized) |
        (original_data['acteurs_2'] == actor_name_normalized) |
        (original_data['actrices'] == actor_name_normalized)
    ]

    print(f"\nFilms avec l'acteur/l'actrice : {actor_name.title()}\n")
    return filtered[['original_title', 'decennie', 'popularity', 'vote_average']]
display(get_films_by_actor("Keanu Reeves"))



Films avec l'acteur/l'actrice : Keanu Reeves



,original_title,decennie,popularity,vote_average
45,The Matrix Revolutions,2000.0,26.049,6.705
128,Sweet November,2000.0,15.020,6.870
372,Something's Gotta Give,2000.0,13.930,6.622
1893,John Wick: Chapter 4,2020.0,1170.178,7.926


In [14]:
def recommend_by_director(index: int,
                          original_data: pd.DataFrame = films,
                          min_popularity: float = None,
                          decade: str = None,
                          sort_by: str = 'popularity',
                          top_n: int = 10) -> pd.DataFrame:
    def normalize_name(name):
        return name.strip().lower().replace("é", "e").replace("-", " ")

    # Nettoyage du nom du réalisateur
    original_data['realisateurs'] = original_data['realisateurs'].fillna('').astype(str).apply(normalize_name)

    film_ref = original_data.iloc[index]
    director_ref = normalize_name(film_ref['realisateurs'])

    # Filtrer films avec le même réalisateur (exact match)
    filtered = original_data[(original_data.index != index) & (original_data['realisateurs'] == director_ref)]

    if min_popularity is not None:
        filtered = filtered[filtered['popularity'] >= min_popularity]

    if decade is not None:
        filtered = filtered[filtered['decennie'] == decade]

    if sort_by and sort_by in filtered.columns:
        filtered = filtered.sort_values(by=sort_by, ascending=False)

    print(f"\nFilms réalisés par {director_ref.title()} :\n")
    print(f"\nRecommandations avec le même réalisateur que « {film_ref['original_title']} » :\n")
    return filtered.head(top_n)

display(recommend_by_director(
    index=44,
    sort_by='popularity',
    min_popularity=None,
    top_n=10
))


Films réalisés par James Cameron :


Recommandations avec le même réalisateur que « Titanic » :



,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
3813,36955,tt0111503,en,True Lies,"A fearless, globe-trotting, terrorist-battling...",32.758,/pweFTnzzTfGK68woSVkiTgjLzWm.jpg,['US'],1994-07-15,"['en', 'fr', 'ar', 'de']",...,7.0,3518,1990.0,1994.0,141.0,"['Action', 'Comedy', 'Thriller']",arnold schwarzenegger,tom arnold,jamie lee curtis,james cameron


In [15]:
films

,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,...,vote_average,vote_count,decennie,startYear,runtimeMinutes,genres,acteurs_1,acteurs_2,actrices,realisateurs
0,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.7700,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",...,6.700,41,1960.0,1962.0,110.0,"['Drama', 'Romance']",jean pierre leaud,patrick auffay,marie france pisier,shintarô ishihara
1,24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.5560,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",...,7.970,15748,2000.0,2003.0,111.0,"['Action', 'Crime', 'Thriller']",david carradine,michael madsen,uma thurman,quentin tarantino
2,76,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.3250,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",...,7.968,3547,1990.0,1995.0,101.0,"['Comedy', 'Drama', 'Romance']",ethan hawke,hanno pöschl,julie delpy,richard linklater
3,80,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.1840,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",...,7.818,2943,2000.0,2004.0,80.0,"['Drama', 'Romance']",ethan hawke,vernon dobtcheff,julie delpy,richard linklater
4,101,tt0110413,en,Léon: The Professional,"Léon, the top hit man in New York, has earned ...",44.7010,/yI6X2cCM5YPJtxMhUd3dPGqDAhw.jpg,"['US', 'FR']",1994-09-14,"['en', 'fr', 'it']",...,8.300,13202,1990.0,1994.0,110.0,"['Action', 'Crime', 'Drama']",jean reno,gary oldman,natalie portman,luc besson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8566,1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,['France'],2025-03-05,['fr'],...,6.600,5,2020.0,2025.0,101.0,"['Drame', 'Romance', 'Comédie']",[',,[,[
8567,892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,['France'],2024-08-11,"['en', 'fr', 'it']",...,5.500,4,2020.0,2024.0,83.0,"['Comédie', 'Drame']",[',,[,[
8568,1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"['Belgium', 'France']",2025-11-12,['fr'],...,0.000,0,2020.0,2025.0,108.0,"['Drame', 'Comédie']",[',,[,[
8569,1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,['France'],2024-05-01,['fr'],...,5.000,1,2020.0,2024.0,98.0,['Drame'],[],,[,[
